In [1]:
import os
if not os.path.exists('../pretrain/params'):
    os.mkdir('../pretrain/params')

# paddle -> numpy

In [1]:
import paddle
import pickle

paddle预训练权重：https://paddleocr.bj.bcebos.com/dygraph_v2.0/pgnet/train_step1.tar

paddle公开最好权重：https://paddleocr.bj.bcebos.com/dygraph_v2.0/pgnet/en_server_pgnetA.tar

下载并存放于 pretrain/paddle/

In [2]:
weight = paddle.load('../pretrain/paddle/pgnet_train_step1.pdparams', return_numpy=True)
# weight = paddle.load('../pretrain/paddle/pgnet_best.pdparams', return_numpy=True)

In [3]:
weight['backbone.bb_0_0.conv0._batch_norm._mean'].shape

(64,)

查看权重名称和形状，结果存放于pretrain/params

In [4]:
with open('../pretrain/params/pgnet_train_step1.txt', 'w') as fw:
# with open('../pretrain/params/pgnet_best.txt', 'w') as fw:
    for key, value in weight.items():
        fw.write(f'{key:50}\t {value.shape}\n')

保存numpy类型的权重，结果存放于pretrain/numpy

In [ ]:
pickle.dump(weight, open('../pretrain/numpy/pgnet_train_step1_param.pkl', 'wb'))
# pickle.dump(weight, open('../pretrain/numpy/pgnet_best_param.pkl', 'wb'))

如果虚拟环境中同时包含paddle和mindspore，则可以直接运行下面的单元格，否则请切换虚拟环境。

# numpy -> mindspore

In [ ]:
import mindspore as ms
import pickle

加载numpy类型权重，以及用于对比的MindSpore权重。

In [4]:
paddle_param = pickle.load(open('../pretrain/numpy/pgnet_train_step1_param.pkl', 'rb'))
# paddle_param = pickle.load(open('../pretrain/numpy/pgnet_best_param.pkl', 'rb'))
mind_param = ms.load_checkpoint('../work_dir/pgnet/e0.ckpt')
# mind_param = ms.load_checkpoint('../work_dir/pgnet/best.ckpt')

查看MindSpore权重名称和形状，结果存放于pretrain/params

In [ ]:
with open('../pretrain/params/ms_pgnet.txt', 'w') as fw:
    for key, value in mind_param.items():
        fw.write(f'{key:50}\t {value.shape}\n')

In [9]:
new_param = []
for key, param in paddle_param.items():
    key_split = key.split('.')
    key_split = [i.strip('_') for i in key_split]

    if key_split[-2] == 'batch_norm' or key_split[-2] == 'bn':
        key_split[-2] = 'norm'
        key_split[-1] = key_split[-1].replace('weight'  , 'gamma')
        key_split[-1] = key_split[-1].replace('bias'    , 'beta')
        key_split[-1] = key_split[-1].replace('mean'    , 'moving_mean')
        key_split[-1] = key_split[-1].replace('variance', 'moving_variance')

    if key_split[0] == 'backbone' and key_split[1].startswith('bb_'):
        _, block, layer = key_split[1].split('_')
        key_split[1] = f'layer{int(block)+1}'
        key_split.insert(2, layer)

    new_key = '.'.join(key_split)
    # 对照
    mind_p = mind_param[new_key]
    assert mind_p.shape == param.shape
    new_param.append({'name': new_key, 'data': ms.Tensor(param, dtype=mind_p.dtype)})

转换后的MindSpore类型权重存放于pretrain/pgnet

In [10]:
ms.save_checkpoint(new_param, '../pretrain/pgnet/pgnet_train_step1.ckpt')
# ms.save_checkpoint(new_param, '../pretrain/pgnet/pgnet_best.ckpt')